In [4]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns

In [8]:
df = pd.read_csv('Raw_Dataset.csv')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796 entries, 0 to 1795
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job ID              1796 non-null   object
 1   Job Title           1796 non-null   object
 2   Company Name        1796 non-null   object
 3   Language and Tools  1629 non-null   object
 4   Job Salary          1239 non-null   object
 5   City                1796 non-null   object
 6   Province            1678 non-null   object
 7   Job Link            1796 non-null   object
dtypes: object(8)
memory usage: 112.4+ KB
None
                 Job ID                                          Job Title  \
0  job_9c7f38160c736c78  Binance Accelerator Program - Data Analyst (Risk)   
1  job_0da15fed6a515fe5                                   Business Analyst   
2  job_2dc0470241aa6066                          Geophysicist/Data Analyst   
3  job_cbbe0e29b236d2b5                Business Intelli

Now that the dataframe is loaded in correctly we can explore the missing values.

In [10]:
print(df.isna().sum())

Job ID                  0
Job Title               0
Company Name            0
Language and Tools    167
Job Salary            557
City                    0
Province              118
Job Link                0
dtype: int64


In [12]:
missing_percentage = df.isna().mean() * 100
print(missing_percentage)

Job ID                 0.000000
Job Title              0.000000
Company Name           0.000000
Language and Tools     9.298441
Job Salary            31.013363
City                   0.000000
Province               6.570156
Job Link               0.000000
dtype: float64


31% (557/1796) of Job Salaries entries are null and numerous entries have special symbols included. There is also discrepancy in the values given, some are in yearly range, some are hourly. I will remove this column as there are too many values to fill and it will not be a reliable data to use. Province missing values are due to City column entry being remote - missing values in this case can be alleviated. Rows with null Language and Tools will be deleted as it provides no information about the job.

In [14]:
#Replacing the null in 'Province' to Remote if City is also Remote
df.loc[df['City'].str.contains('Remote', case=False, na=False), 'Province'] = 'Remote'

In [16]:
# Checking if the replacement was successful
print(df[df['Province'] == 'Remote'][['City', 'Province']].head())


      City Province
0   Remote   Remote
1   Remote   Remote
35  Remote   Remote
36  Remote   Remote
39  Remote   Remote


Seems like the replacement is successfull. Let's check how many rows that are null are left.

In [18]:
print(df.isna().sum())

Job ID                  0
Job Title               0
Company Name            0
Language and Tools    167
Job Salary            557
City                    0
Province               18
Job Link                0
dtype: int64


Job Link column does not provide useful information and has a high probability of being obsolete after some time.                                                                                                 

In [20]:
del df['Job Link']

In [22]:
# Display rows where 'Province' is null
null_province_rows = df[df['Province'].isna()]
print(null_province_rows)

             Job ID                                          Job Title  \
1023  1009308668249                                   CATEGORY ANALYST   
1045  1009389771243                              Data Business Analyst   
1085  1009282875220                Data Quality and Management Analyst   
1088  1009376310781                              business data analyst   
1120  1009384459609  Marketing Data Analyst/Analyste des donnÃƒÆ’Ã‚...   
1148  1009382475450                               Intermediate Analyst   
1230  1009376154022  Business intelligence analyst - Information te...   
1249  1009350023073  Expert Data Warehouse / Data Engineer / Data A...   
1304  1009388363698              Principal Services Operations Analyst   
1311  1009363378192                             Marketing Data Analyst   
1314  1007956853540                                Senior Data Analyst   
1368  1009387787093              Principal Business Continuity Analyst   
1376  1009350830411                   

Exploring those missing values shows a new problem - some Cities are replaced by 'engineer' which is definitely not a city. 'Canada' is also not applicable as a city. 

In [24]:
del df['Job Salary']

In 'City' column Montreal and Boye d'Urfe have special characters that have to be replaced with 'e'.

In [28]:
# Replace the special characters with 'e' in the 'City' column
df['City'] = df['City'].str.replace(r'ÃƒÂ©', 'e', regex=True)
df['City'] = df['City'].str.replace(r'ÃƒÆ’Ã‚Â©', 'e', regex=True)
# Check if the replacement was successful
print(df['City'].unique())


['Remote' 'Ottawa' 'Toronto' 'Niagara' 'Abbotsford' 'Windsor' 'Vancouver'
 'Burlington' 'Calgary' 'Barrie' 'Richmond Hill' 'Weyburn' 'Mississauga'
 'Banff' 'Edmonton' 'Greater Toronto Area' 'Montreal'
 'Remote in Beauceville' 'London' 'Remote in Charlottetown'
 'Saint-Laurent' 'Fredericton' 'Bedford' 'Remote in Toronto' 'Brampton'
 'Vaughan' 'Surrey' 'Red Lake' 'Winnipeg' 'Laval' 'Halifax' 'Dieppe'
 'Vernon' 'Dorval' 'Bolton' 'Sherbrooke' 'Victoria' 'North York'
 'Oakville' 'Richmond' 'Burnaby' 'Metro Vancouver Regional District'
 'Berwick' 'Remote in Moncton' 'Remote in Mount Pearl' 'Remote in Milton'
 'Remote in Boucherville' 'Remote in Woodstock' 'Remote in Ottawa'
 'Remote in Kelowna' 'Remote in Lakeside' 'Remote in Mississauga'
 'Remote in Winnipeg' 'Remote in Regina' 'Remote in Victoria'
 'Remote in Port Coquitlam' 'Remote in Thunder Bay' 'Remote in Calgary'
 'Squamish' 'Remote in Edmonton' 'North Vancouver'
 'Remote in South Dundas' 'Hamilton' 'Waterloo' 'Waterdown' 'Saskatoon'


We can create a new column that contains the Type of work: Remote, Hybrid or In person based on 'City' column.